In [1]:
import pandas as pd
import numpy as np
import surprise 

#1. Read and explore the dataset. (Rename column, plot histograms, find data characteristics)
#2. Take subset of dataset to make it less sparse/more dense. (For example, keep the users only who has given 50 or more number of ratings )
#3. Split the data randomly into train and test dataset. (For example split it in 70/30 ratio)
#4. Build Popularity Recommender model.

In [2]:
elect_ratings = pd.read_csv('ratings_Electronics.csv',header=None)

In [3]:
elect_ratings.head(10)

,0,1,2,3
0,AKM1MP6P0OYPR,0132793040,5.0,1365811200
1,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
2,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
3,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
4,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200
5,A1QGNMC6O1VW39,0511189877,5.0,1397433600
6,A3J3BRHTDRFJ2G,0511189877,2.0,1397433600
7,A2TY0BTJOTENPG,0511189877,5.0,1395878400
8,A34ATBPOK6HCHY,0511189877,5.0,1395532800
9,A89DO69P0XZ27,0511189877,5.0,1395446400


In [4]:
elect_ratings.columns = ['userId','productId','ratings','timestamp']

In [5]:
elect_ratings.head()

,userId,productId,ratings,timestamp
0,AKM1MP6P0OYPR,0132793040,5.0,1365811200
1,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
2,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
3,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
4,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200


In [6]:
elect_ratings.count()

userId       7824482
productId    7824482
ratings      7824482
timestamp    7824482
dtype: int64

In [7]:
elect_ratings=elect_ratings.drop('timestamp',axis=1)

In [8]:
elect_ratings.head(10)

,userId,productId,ratings
0,AKM1MP6P0OYPR,0132793040,5.0
1,A2CX7LUOHB2NDG,0321732944,5.0
2,A2NWSAGRHCP8N5,0439886341,1.0
3,A2WNBOD3WNDNKT,0439886341,3.0
4,A1GI0U4ZRJA8WN,0439886341,1.0
5,A1QGNMC6O1VW39,0511189877,5.0
6,A3J3BRHTDRFJ2G,0511189877,2.0
7,A2TY0BTJOTENPG,0511189877,5.0
8,A34ATBPOK6HCHY,0511189877,5.0
9,A89DO69P0XZ27,0511189877,5.0


In [9]:
print("Unique users in ratings - ",len(elect_ratings["userId"].unique()))

Unique users in ratings -  4201696


In [10]:
no_of_users = elect_ratings['userId'].value_counts().rename('user_counts')

elect_ratings = elect_ratings.merge(no_of_users.to_frame(),
                                left_on='userId',
                                right_index=True)

In [11]:
elect_ratings.sample(10)

,userId,productId,ratings,user_counts
6024415,A1E2E7SBZ5857M,B00825BZUY,4.0,1
3993214,A92OVWI11O64K,B004AUK9IY,5.0,2
50592,AZ4352XO4HH3M,B00004TLPE,5.0,48
4841962,A3UYETHCGUWZ0H,B005DCCNYG,3.0,1
826512,A34LBPE10XYS10,B000E0I58K,1.0,4
1359628,A39XJJ6RAVV9X7,B000SMZDIY,5.0,17
3260607,A8ECEQD9BS5F4,B003DT4QM8,5.0,2
3063589,A1UDIASMHQLP3N,B0035ER1LE,5.0,15
917407,A130SQEUGQJLKC,B000F9YN22,4.0,1
625531,A1BZQ21I3OB3MG,B0007WK8KS,5.0,2


In [35]:
elect_ratings=elect_ratings[elect_ratings.user_counts > 50]

In [36]:
elect_ratings.sample(10)

,userId,productId,ratings,user_counts
503813,A2LXX47A0KMJVX,B0002RQU6E,1.0,91
2178344,A22CW0ZHY3NJH8,B001NPDA44,4.0,292
7189831,A1V3TRGWOMA8LC,B00BXF7I9M,5.0,101
3366073,A1X2LENOF84LCQ,B003H3HLEU,2.0,86
2615806,AGOH8N902URMW,B002HWRJBM,5.0,131
824443,A25FL6VLD7S23S,B000DZH4D8,5.0,111
4024694,A3R1WFO4KIUQXZ,B004CLYEE6,4.0,134
6334598,A37CEYB95LK6R6,B008PO5CSI,5.0,98
582020,A22I55P15NSAOX,B00077AA5Q,5.0,90
4421469,ADOR3TR7GDF68,B004SBBD8U,5.0,111


In [37]:
elect_ratings.count()

userId         61106
productId      61106
ratings        61106
user_counts    61106
dtype: int64

In [38]:
X=elect_ratings.drop('ratings',axis=1)

In [39]:
y=elect_ratings['ratings']

In [40]:
X.head()

,userId,productId,user_counts
118,AT09WGFUM934H,0594481813,110
77282,AT09WGFUM934H,B00005105L,110
160734,AT09WGFUM934H,B000068O1M,110
160973,AT09WGFUM934H,B000068O34,110
162248,AT09WGFUM934H,B000068O4J,110


In [41]:
y.head()

118       3.0
77282     5.0
160734    5.0
160973    5.0
162248    5.0
Name: ratings, dtype: float64

In [42]:
X.shape

(61106, 3)

In [43]:
y.shape

(61106,)

In [44]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,random_state=10)

In [45]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(42774, 3)
(18332, 3)
(42774,)
(18332,)


5. Build Collaborative Filtering model.
6. Evaluate both the models. (Once the model is trained on the training data, it can be used to compute the error (RMSE) on predictions made on the test data.)
7. Get top - K (K = 5) recommendations. Since our goal is to recommend new products to each user based on his/her habits,we will recommend 5 new products.
8. Summarise your insights.

In [46]:
from surprise import Dataset,Reader
reader = Reader(rating_scale=(1, 5))

In [47]:
data = Dataset.load_from_df(elect_ratings[['userId','productId','ratings']], reader)

In [48]:
data

In [49]:
# Split data to train and test
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=.25,random_state=74)

# to build on full data
#trainset = data.build_full_trainset()

In [50]:
type(trainset)

surprise.trainset.Trainset

In [51]:
user_records = trainset.ur
type(user_records)

collections.defaultdict

In [52]:
user_records[0]

[(0, 2.0),
 (122, 5.0),
 (134, 4.0),
 (361, 4.0),
 (2112, 4.0),
 (3414, 5.0),
 (3448, 4.0),
 (1492, 5.0),
 (5076, 4.0),
 (4072, 5.0),
 (5631, 4.0),
 (5645, 5.0),
 (168, 4.0),
 (6447, 5.0),
 (6650, 5.0),
 (6740, 4.0),
 (7181, 4.0),
 (7592, 3.0),
 (7858, 5.0),
 (1517, 4.0),
 (8501, 4.0),
 (3307, 4.0),
 (8850, 4.0),
 (9469, 5.0),
 (1618, 4.0),
 (9648, 3.0),
 (9715, 5.0),
 (5957, 4.0),
 (10233, 3.0),
 (10469, 5.0),
 (10584, 5.0),
 (7529, 5.0),
 (12414, 3.0),
 (12849, 4.0),
 (848, 4.0),
 (4052, 5.0),
 (13165, 5.0),
 (1317, 3.0),
 (1384, 5.0),
 (13668, 5.0),
 (8339, 5.0),
 (11464, 5.0),
 (14882, 5.0),
 (15088, 1.0),
 (15114, 4.0),
 (15801, 3.0),
 (16114, 4.0),
 (796, 5.0),
 (17143, 4.0),
 (17151, 1.0),
 (17215, 4.0),
 (1950, 3.0),
 (1369, 4.0),
 (18000, 4.0),
 (18216, 5.0),
 (11211, 5.0),
 (19038, 5.0),
 (4719, 5.0),
 (19317, 5.0),
 (15274, 4.0),
 (1416, 5.0),
 (19699, 5.0),
 (19737, 4.0),
 (536, 4.0),
 (449, 5.0),
 (3873, 5.0),
 (3147, 5.0),
 (3665, 5.0),
 (21791, 4.0),
 (11413, 3.0),
 (186

In [53]:
from surprise import KNNWithMeans
from surprise import accuracy

In [57]:
algo = KNNWithMeans(k=20, sim_options={})
# Used default default sim options ==> pearson takes lot of RAM and system crashes due to Memory Error...

In [58]:
algo.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [59]:
len(testset)

15277

In [60]:
testset[0:5]

[('AGGAQLXPVGM5D', 'B000P2ZPB2', 5.0),
 ('A1KXJCXS6HFRQZ', 'B00BQ5RY1G', 4.0),
 ('A37CEYB95LK6R6', 'B0000512E1', 4.0),
 ('A11EXFO14WEJM1', 'B00F5NJSPK', 5.0),
 ('A1TDR7Y90SVCHL', 'B00004SABB', 3.0)]

In [63]:
# Evalute on test set
test_predict = algo.test(testset)

# compute RMSE
accuracy.rmse(test_predict)

RMSE: 1.0378


1.037811898388877

###### Generating top n recommendations

In [64]:
testset_new = trainset.build_anti_testset()

In [65]:
len(testset_new)

11182889

In [66]:
testset_new[0:5]

[('AMVXR3NPL0GWX', 'B00B0RT3Y6', 4.270920159724192),
 ('AMVXR3NPL0GWX', 'B0013FSKSA', 4.270920159724192),
 ('AMVXR3NPL0GWX', 'B000E5GGQI', 4.270920159724192),
 ('AMVXR3NPL0GWX', 'B005ZRNOS8', 4.270920159724192),
 ('AMVXR3NPL0GWX', 'B00G6CLNCK', 4.270920159724192)]

In [67]:
predictions = algo.test(testset_new[0:100000])

In [68]:
predictions

[Prediction(uid='AMVXR3NPL0GWX', iid='B00B0RT3Y6', r_ui=4.270920159724192, est=3.694146341463415, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid='AMVXR3NPL0GWX', iid='B0013FSKSA', r_ui=4.270920159724192, est=5, details={'actual_k': 2, 'was_impossible': False}),
 Prediction(uid='AMVXR3NPL0GWX', iid='B000E5GGQI', r_ui=4.270920159724192, est=4.449596602972399, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid='AMVXR3NPL0GWX', iid='B005ZRNOS8', r_ui=4.270920159724192, est=5, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid='AMVXR3NPL0GWX', iid='B00G6CLNCK', r_ui=4.270920159724192, est=3.447786686909529, details={'actual_k': 6, 'was_impossible': False}),
 Prediction(uid='AMVXR3NPL0GWX', iid='B001RB1XK2', r_ui=4.270920159724192, est=4.226666666666667, details={'actual_k': 0, 'was_impossible': False}),
 Prediction(uid='AMVXR3NPL0GWX', iid='B004T9Y0MW', r_ui=4.270920159724192, est=1.418974358974359, details={'actual_k': 1, 'was_impossible'

In [69]:
predictions_df = pd.DataFrame([[x.uid,x.iid,x.est] for x in predictions])

In [70]:
predictions_df.columns = ["userId","productId","ratings"]
predictions_df.sort_values(by = ["userId", "ratings","productId"],ascending=False,inplace=True)

In [71]:
predictions_df.head()

,userId,productId,ratings
5850,AMVXR3NPL0GWX,B00KHA5G6G,5.0
12442,AMVXR3NPL0GWX,B00K9NN1VS,5.0
18067,AMVXR3NPL0GWX,B00K5IVF1K,5.0
22469,AMVXR3NPL0GWX,B00K2LMVJK,5.0
9384,AMVXR3NPL0GWX,B00JL98TRC,5.0


In [72]:
top_5_recos = predictions_df.groupby("userId").head(5).reset_index(drop=True)

In [74]:
top_5_recos.head(20)

,userId,productId,ratings
0,AMVXR3NPL0GWX,B00KHA5G6G,5.0
1,AMVXR3NPL0GWX,B00K9NN1VS,5.0
2,AMVXR3NPL0GWX,B00K5IVF1K,5.0
3,AMVXR3NPL0GWX,B00K2LMVJK,5.0
4,AMVXR3NPL0GWX,B00JL98TRC,5.0
5,AD9ZL63H0VKCZ,B00KGTDDIG,5.0
6,AD9ZL63H0VKCZ,B00HFW1E1A,5.0
7,AD9ZL63H0VKCZ,B00H4OWPE4,5.0
8,AD9ZL63H0VKCZ,B00FRDUZUK,5.0
9,AD9ZL63H0VKCZ,B00ESYW40S,5.0
